In [0]:
import pandas as pd
import json 
from tqdm.notebook import tqdm
from collections import Counter, defaultdict
import numpy as np

In [0]:
neg = pd.read_csv('negex.csv', index_col = 0)
pos = pd.read_csv('posex.csv', index_col = 0)

In [0]:
def splitter(sent):
    splsent = []
    sent = sent.split('\n')
    for word in sent:
        newword = word.split('\t')
        splsent.append(newword)
    return splsent

In [0]:
neg['token'] = neg['token'].apply(splitter)
negsents = neg['token'].tolist()
neg_verbs = []
neg_ex = []
for i in tqdm(range(len(negsents))):
    for word in negsents[i]:
        if word[3] == 'VERB':
            neg_verbs.append(word)
            neg_ex.append(neg['untoken'][i])

In [0]:
pos['token'] = pos['token'].apply(splitter)
possents = pos['token'].tolist()
pos_verbs = []
pos_ex = []
for i in tqdm(range(len(possents))):
    for word in possents[i]:
        if word[3] == 'VERB':
            pos_verbs.append(word)
            pos_ex.append(pos['untoken'][i])


In [0]:
neg_inf_verbs = []
for word in neg_verbs:
    neg_inf_verbs.append(word[2])
pos_inf_verbs = []
for word in pos_verbs:
    pos_inf_verbs.append(word[2])

In [0]:
un_verbs = list(set(neg_inf_verbs) | set(pos_inf_verbs))

In [0]:
len(set(pos_inf_verbs))

14956

In [0]:
len(set(neg_inf_verbs))

14335

In [0]:
len(un_verbs)

22404

In [0]:
neg_verbs_dict = Counter(neg_inf_verbs)
pos_verbs_dict = Counter(pos_inf_verbs)

In [0]:
verb_df = pd.DataFrame(columns = ['neg_entries', 'pos_entries'], index = un_verbs)
for i in tqdm(range(len(un_verbs))):
    verb_df['neg_entries'][un_verbs[i]] = neg_verbs_dict[un_verbs[i]]
    verb_df['pos_entries'][un_verbs[i]] = pos_verbs_dict[un_verbs[i]]

verb_df['summa'] = verb_df['neg_entries']+verb_df['pos_entries']

In [0]:
verb_df = verb_df.sort_values(by = 'summa', ascending = False)
verb_df = verb_df.loc[verb_df['summa'] > 50]
cleaned_verbs = verb_df.index.values

In [0]:
len(cleaned_verbs)

981

In [0]:
verb_asp = defaultdict(list)
verb_ex = defaultdict(list)

for i in tqdm(range(len(cleaned_verbs))):
    for sent in negsents:
        for word in sent:
            if word[2] == cleaned_verbs[i]:
                verb_asp[word[2]].append(word[5])
                verb_ex[word[2]].append(sent)

    for sent in possents:
        for word in sent:
            if word[2] == cleaned_verbs[i]:
                verb_asp[word[2]].append(word[5])
                verb_ex[word[2]].append(sent)


In [0]:
asp_df = pd.DataFrame(columns = ['aspects', 'examples'], index = cleaned_verbs)
for verb in cleaned_verbs:
    asp_df['aspects'][verb] = verb_asp[verb]
    asp_df['examples'][verb] = verb_ex[verb]

In [0]:
from google.colab import drive
drive.mount('/drive')
verb_df.to_csv('/drive/My Drive/Colab Notebooks/verb_df.csv')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [0]:
def asp_extractor(dframe):
    list_of_verbs = dframe.index.values
    aspects = dframe['aspects'].tolist()
    numbdict = defaultdict(list)
    tensedict = defaultdict(list)
    genddict = defaultdict(list)
    voicedict = defaultdict(list)
    persondict = defaultdict(list)
    for i in tqdm(range(len(aspects))):
        for z in range(len(aspects[i])):
            singasp = aspects[i][z].split("|")
            for item in singasp:
                new_i = item.split("=")
                if new_i[0] == 'Number':
                    numbdict[list_of_verbs[i]].append(new_i[1])
                if new_i[0] == 'Tense':
                    tensedict[list_of_verbs[i]].append(new_i[1])
                if new_i[0] == 'Gender':
                    genddict[list_of_verbs[i]].append(new_i[1])
                if new_i[0] == 'Voice':
                    voicedict[list_of_verbs[i]].append(new_i[1])
                if new_i[0] == 'Person':
                    persondict[list_of_verbs[i]].append(new_i[1])
            if len(numbdict[list_of_verbs[i]]) != z+1:
                numbdict[list_of_verbs[i]].append(None)
            if len(tensedict[list_of_verbs[i]]) != z+1:
                tensedict[list_of_verbs[i]].append(None)
            if len(genddict[list_of_verbs[i]]) != z+1:
                genddict[list_of_verbs[i]].append(None)
            if len(voicedict[list_of_verbs[i]]) != z+1:
                voicedict[list_of_verbs[i]].append(None)
            if len(persondict[list_of_verbs[i]]) != z+1:
                persondict[list_of_verbs[i]].append(None)
            

    return numbdict, tensedict, genddict, voicedict, persondict

In [0]:
numbdict, tensedict, genddict, voicedict, persondict = asp_extractor(asp_df)

In [0]:
verb_df['sing'] = None
verb_df['plur'] = None
verb_df['none_numb'] = None
verb_df['fut'] = None
verb_df['pres'] = None
verb_df['past'] = None
verb_df['none_tense'] = None
verb_df['1per'] = None
verb_df['2per'] = None
verb_df['3per'] = None
verb_df['none_per'] = None
verb_df['mid'] = None
verb_df['act'] = None
verb_df['pass'] = None
verb_df['none_voice'] = None
verb_df['masc'] = None
verb_df['fem'] = None
verb_df['neut'] = None
verb_df['none_gend'] = None

In [0]:
voicedict['хотеть']

In [0]:
for verb in cleaned_verbs:
    c = Counter(numbdict[verb])
    verb_df['sing'][verb] = c['Sing']/len(numbdict[verb])*100
    verb_df['plur'][verb] = c['Plur']/len(numbdict[verb])*100
    verb_df['none_numb'][verb] = c[None]/len(numbdict[verb])*100
    c = Counter(tensedict[verb])
    verb_df['pres'][verb] = c['Pres']/len(tensedict[verb])*100
    verb_df['past'][verb] = c['Past']/len(tensedict[verb])*100
    verb_df['fut'][verb] = c['Fut']/len(tensedict[verb])*100
    verb_df['none_tense'][verb] = c[None]/len(tensedict[verb])*100
    c = Counter(genddict[verb])
    verb_df['masc'][verb] = c['Masc']/len(genddict[verb])*100
    verb_df['fem'][verb] = c['Fem']/len(genddict[verb])*100
    verb_df['neut'][verb] = c['Neut']/len(genddict[verb])*100
    verb_df['none_gend'][verb] = c[None]/len(genddict[verb])*100
    c = Counter(voicedict[verb])
    verb_df['mid'][verb] = c['Mid']/len(voicedict[verb])*100
    verb_df['act'][verb] = c['Act']/len(voicedict[verb])*100
    verb_df['pass'][verb] = c['Pass']/len(voicedict[verb])*100
    verb_df['none_voice'][verb] = c[None]/len(voicedict[verb])*100
    c = Counter(persondict[verb])
    verb_df['1per'][verb] = c['1']/len(persondict[verb])*100
    verb_df['2per'][verb] = c['2']/len(persondict[verb])*100
    verb_df['3per'][verb] = c['3']/len(persondict[verb])*100
    verb_df['none_per'][verb] = c[None]/len(persondict[verb])*100

In [0]:
verb_df

,neg_entries,pos_entries,summa,sing,plur,none_numb,fut,pres,past,none_tense,1per,2per,3per,none_per,mid,act,pass,none_voice,masc,fem,neut,none_gend
быть,6135,7283,13418,79.8627,12.8524,7.28481,0.422308,54.3659,36.688,8.52379,15.672,3.29152,36.9054,44.1312,0.00621041,99.6367,0,0.357098,8.52068,7.35623,17.5102,66.6128
хотеть,7856,3572,11428,92.9352,5.98916,1.07564,0.0516307,80.4234,17.2963,2.22872,63.7553,6.54849,10.1885,19.5078,0,98.3392,0,1.66079,6.00637,10.8855,0.0430256,83.0651
мочь,5333,3625,8958,89.5134,10.0987,0.387984,0,88.2607,11.0076,0.731626,46.3918,6.40727,35.4728,11.7282,0,99.3016,0,0.69837,3.43643,5.02162,0.753797,90.7882
знать,3289,3497,6786,82.8349,10.774,6.39109,0,81.4277,11.5509,7.0214,52.3307,15.7725,13.3832,18.5136,0,99.4723,0,0.527704,3.89915,6.40575,0,89.6951
любить,2343,3468,5811,83.2655,11.0885,5.64598,0,88.9115,4.10309,6.98542,65.3442,7.15497,12.0888,15.412,0,94.354,4.17091,1.47508,3.22143,3.91658,0.0508647,92.8111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
предупреждать,21,30,51,62.7451,17.6471,19.6078,0,37.2549,35.2941,27.451,19.6078,7.84314,17.6471,54.902,0,100,0,0,13.7255,15.6863,0,70.5882
чуять,33,18,51,100,0,0,0,88.2353,11.7647,0,72.549,0,15.6863,11.7647,0,100,0,0,7.84314,3.92157,0,88.2353
лопнуть,27,24,51,69.8113,24.5283,5.66038,54.717,3.77358,33.9623,7.54717,22.6415,0,35.8491,41.5094,54.717,41.5094,0,3.77358,16.9811,7.54717,3.77358,71.6981
палить,19,32,51,38.8889,27.7778,33.3333,11.1111,35.1852,14.8148,38.8889,14.8148,7.40741,24.0741,53.7037,0,94.4444,0,5.55556,3.7037,11.1111,1.85185,83.3333


In [0]:
from google.colab import drive
drive.mount('/drive')
verb_df.to_csv('/drive/My Drive/Colab Notebooks/verb_df_2.csv')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [0]:
verb_df = pd.read_csv('verb_df_2.csv', index_col = 0)

In [0]:
verbs = verb_df.index.values

In [0]:
subjdict = defaultdict(list)
objdict = defaultdict(list)
for i in tqdm(range(len(verbs))):
    sents = verb_ex[verbs[i]]
    for sent in sents:
        for word in sent:
            if word[7] == 'nsubj':
                subjdict[verbs[i]].append(word[2])
            if word[7] == 'obj':
                objdict[verbs[i]].append(word[2])


In [0]:
objects_list = []
fsubj_list = []
ssubj_list = []
for verb in verbs:
    objects = Counter(objdict[verb]).most_common(1)
    if len(objects) != 0:
        objects_list.append(objects[0][0])
    else:
        objects_list.append(None)
    subjects = Counter(subjdict[verb]).most_common(2)
    if len(subjects) != 0:
        fsubj_list.append(subjects[0][0])
        ssubj_list.append(subjects[1][0])

In [0]:
verb_df['object'] = objects_list
verb_df['first_subj'] = fsubj_list
verb_df['second_subj'] = ssubj_list

In [0]:
verb_df

,neg_entries,pos_entries,summa,sing,plur,none_numb,fut,pres,past,none_tense,1per,2per,3per,none_per,mid,act,pass,none_voice,masc,fem,neut,none_gend,object,first_subj,second_subj
быть,6135,7283,13418,79.862750,12.852441,7.284809,0.422308,54.365917,36.687989,8.523786,15.671966,3.291517,36.905353,44.131164,0.006210,99.636691,0.000000,0.357098,8.520681,7.356229,17.510247,66.612843,я,я,@
хотеть,7856,3572,11428,92.935204,5.989158,1.075639,0.051631,80.423371,17.296274,2.228724,63.755271,6.548490,10.188452,19.507788,0.000000,98.339213,0.000000,1.660787,6.006368,10.885466,0.043026,83.065141,я,я,ты
мочь,5333,3625,8958,89.513358,10.098659,0.387984,0.000000,88.260725,11.007649,0.731626,46.391753,6.407272,35.472786,11.728190,0.000000,99.301630,0.000000,0.698370,3.436426,5.021616,0.753797,90.788161,я,я,@
знать,3289,3497,6786,82.834946,10.773967,6.391088,0.000000,81.427734,11.550865,7.021401,52.330695,15.772501,13.383172,18.513632,0.000000,99.472296,0.000000,0.527704,3.899150,6.405746,0.000000,89.695104,что,я,ты
любить,2343,3468,5811,83.265514,11.088505,5.645982,0.000000,88.911495,4.103086,6.985419,65.344184,7.154968,12.088844,15.412004,0.000000,94.354018,4.170905,1.475076,3.221431,3.916582,0.050865,92.811122,ты,я,ты
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
предупреждать,21,30,51,62.745098,17.647059,19.607843,0.000000,37.254902,35.294118,27.450980,19.607843,7.843137,17.647059,54.901961,0.000000,100.000000,0.000000,0.000000,13.725490,15.686275,0.000000,70.588235,ты,я,@
чуять,33,18,51,100.000000,0.000000,0.000000,0.000000,88.235294,11.764706,0.000000,72.549020,0.000000,15.686275,11.764706,0.000000,100.000000,0.000000,0.000000,7.843137,3.921569,0.000000,88.235294,я,я,выходной
лопнуть,27,24,51,69.811321,24.528302,5.660377,54.716981,3.773585,33.962264,7.547170,22.641509,0.000000,35.849057,41.509434,54.716981,41.509434,0.000000,3.773585,16.981132,7.547170,3.773585,71.698113,они,я,@
палить,19,32,51,38.888889,27.777778,33.333333,11.111111,35.185185,14.814815,38.888889,14.814815,7.407407,24.074074,53.703704,0.000000,94.444444,0.000000,5.555556,3.703704,11.111111,1.851852,83.333333,я,я,он


In [0]:
f = open('frames.json',)
data = json.load(f)

elements = ['0','1']
variants = []
effect = []
polarity = []
state = []
value = []
rolea0 = []
rolea1 = []
rolea2 = []
title = []

for el in elements:
    for i in range(200):
        name = el + '_' + str(i)
        if name in data:
          if 'effect' in data[name]['frames']:
              effect.append(data[name]['frames']['effect'])
          else:
              effect.append(None)

          if 'polarity' in data[name]['frames']:
              polarity.append(data[name]['frames']['polarity'])
          else:
              polarity.append(None)

          if 'state' in data[name]['frames']:
              state.append(data[name]['frames']['state'])
          else:
              state.append(None)

          if 'value' in data[name]['frames']:
              value.append(data[name]['frames']['value'])
          else:
              value.append(None)

          if 'a0' in data[name]['roles']:
              rolea0.append(data[name]['roles']['a0'])
          else:
              rolea0.append(None)

          if 'a1' in data[name]['roles']:
              rolea1.append(data[name]['roles']['a1'])
          else:
              rolea1.append(None)

          if 'a2' in data[name]['roles']:
              rolea2.append(data[name]['roles']['a2'])
          else:
              rolea2.append(None)

          title.append(data[name]['title'])

          variants.append(data[name]['variants'])

extrdata = {'variants' : variants,
            'title' : title,
            'role_a0' : rolea0,
            'role_a1' : rolea1,
            'role_a2' : rolea2,
            'value' : value,
            'state' : state,
            'polarity' : polarity,
            'effect' : effect}
framework = pd.DataFrame(data = extrdata)

In [0]:
frverbs = framework['variants'].tolist()
merge = lambda ll: [el for lst in ll for el in lst]
mergedfrverbs = list(set(merge(frverbs)))

In [0]:
intersection_verbs = list(set(verbs) & set(mergedfrverbs))

In [0]:
interpos = list(set(pos_inf_verbs) & set(mergedfrverbs))

In [0]:
interneg = list(set(neg_inf_verbs) & set(mergedfrverbs))

In [0]:
len(list(set(interpos) & set(interneg)))

1486

In [0]:
verb_df.columns.values

array(['neg_entries', 'pos_entries', 'summa', 'sing', 'plur', 'none_numb',
       'fut', 'pres', 'past', 'none_tense', '1per', '2per', '3per',
       'none_per', 'mid', 'act', 'pass', 'none_voice', 'masc', 'fem',
       'neut', 'none_gend'], dtype=object)

In [0]:
verb_df['neg_entries'] = verb_df['neg_entries']/verb_df['summa']*100
verb_df['pos_entries'] = verb_df['pos_entries']/verb_df['summa']*100

In [0]:
verb_df = verb_df.drop(columns = ['summa'])

In [0]:
temp_df = verb_df.T
train_data = pd.DataFrame(columns = verb_df.columns.values)
for verb in intersection_verbs:
    train_data.loc[verb] = temp_df[verb].tolist()
verb_df = verb_df.drop(intersection_verbs, axis = 0)

In [0]:
counter = []
effects = defaultdict(list)
for i in tqdm(range(len(frverbs))):
    for verb in frverbs[i]:
        if verb in intersection_verbs and verb not in counter:
            effects[verb].append(framework[i:i+1]['effect'].tolist())
        counter.append(verb)

In [0]:
a0_list = []
a1_list = []
for i in tqdm(range(len(intersection_verbs))):
    if intersection_verbs[i] in list(effects.keys()):
        k = effects[intersection_verbs[i]][0][0]
        a0 = 0
        a1 = 0
        if k == None:
            a0_list.append(a0)
            a1_list.append(a1)
        else:
            for ef in k:
                if ef[0] == 'a0':
                    if ef[1] == '-':
                        a0 = ef[2] * -1
                    else:
                        a0 = ef[2]
                if ef[0] == 'a1':
                    if ef[1] == '-':
                        a1 = ef[2] * -1
                    else:
                        a1 = ef[2]
            a0_list.append(a0)
            a1_list.append(a1)

In [0]:
train_data['a0_effect'] = a0_list
train_data['a1_effect'] = a1_list

In [0]:
verb_df.to_csv('/drive/My Drive/Colab Notebooks/test_data_2.csv')
train_data.to_csv('/drive/My Drive/Colab Notebooks/train_data_2.csv')

NameError: ignored

In [0]:
!wget http://vectors.nlpl.eu/repository/20/180.zip
!unzip 180.zip

--2020-05-30 14:35:30--  http://vectors.nlpl.eu/repository/20/180.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.225
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.225|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 484452317 (462M) [application/zip]
Saving to: ‘180.zip’

180.zip             100%[===================>] 462.01M  21.2MB/s    in 24s     

2020-05-30 14:35:55 (18.9 MB/s) - ‘180.zip’ saved [484452317/484452317]

Archive:  180.zip
  inflating: meta.json               
  inflating: model.bin               
  inflating: model.txt               
  inflating: README                  


In [0]:
test = pd.read_csv('test_data_2.csv', index_col = 0)
train = pd.read_csv('train_data_2.csv', index_col = 0)

In [0]:
o = train['object'].tolist()
fs = train['first_subj'].tolist()
ss = train['second_subj'].tolist()

In [0]:
all = list(set(o) | set(fs) | set(ss))
Counter(all)

In [0]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format("model.bin", binary=True)  # C bin format

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
entries = defaultdict(list)
for i in model.wv.vocab.keys():
   entry = i.split('_')
   if entry[0] in all:
      entries[entry[0]].append(model.wv[i])


In [0]:
for verb in train.index.values:
    new_verb = verb+'_'+'VERB'
    if new_verb in model.wv.vocab:
        print(True)

In [0]:
new_obj = []
for word in o:
    if len(entries[word]) !=0:
        new_obj.append(entries[word][0])
    else:
        new_obj.append(np.zeros(4575))
new_fs = []
for word in fs:
    if len(entries[word]) !=0:
        new_fs.append(entries[word][0])
    else:
        new_fs.append(np.zeros(4575))
new_ss = []
for word in ss:
    if len(entries[word]) !=0:
        new_ss.append(entries[word][0])
    else:
        new_ss.append(np.zeros(4575))

In [0]:
train['object'] = new_obj
train['first_subj'] = new_fs
train['second_subj'] = new_ss

In [0]:
train.to_csv('/drive/My Drive/Colab Notebooks/train_data_2.csv')

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from imblearn.over_sampling import RandomOverSampler


from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier

In [0]:
def predictor_v2(model, X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
    modulation = model.fit(X_train, y_train)
    preds = model.predict(X_test)
    z=0
    for i in range(len(y_test)):
        if y_test[i].all() == preds[i].all():
            z += 1
    print("Accuracy score: ", z/len(y_test))
    return y_test, preds

In [0]:
def to_arr (l):
  l = np.array([l])
  return l

In [0]:
test = pd.read_csv('test_data_2.csv', index_col = 0)
#test = test.drop(columns = ['summa'])
train = pd.read_csv('train_data_2.csv', index_col = 0)
train['a0_effect'] = train.apply(lambda x: round(x['a0_effect'], 0), axis = 1)
#train = train.drop(columns = ['sing','plur','none_numb'])
train = train.drop(columns = ['fem','masc','neut', 'none_gend'])
train = train.drop(columns = ['past','pres','fut','none_tense'])
#train = train.drop(columns = ['1per','2per','3per','none_per'])
#train = train.drop(columns = ['mid','act','pass','none_voice'])
y = train[['a0_effect',	"a1_effect"]].to_numpy()
test = test.drop(columns = ['object','first_subj','second_subj'])
test = test.drop(columns = ['fem','masc','neut', 'none_gend'])
test = test.drop(columns = ['past','pres','fut','none_tense'])
XT = train.drop(columns = ['a0_effect',	"a1_effect",'object','first_subj','second_subj'])

In [0]:
ob = train['object'].apply(to_arr).tolist()
fsu = train['first_subj'].apply(to_arr).tolist()
ssu = train['second_subj'].apply(to_arr).tolist()

In [0]:
XT_new = train['object'].apply(to_arr)

In [0]:
y = train[["a1_effect"]]

In [0]:
XT.columns.values

array(['neg_entries', 'pos_entries', 'sing', 'plur', 'none_numb', '1per',
       '2per', '3per', 'none_per', 'mid', 'act', 'pass', 'none_voice'],
      dtype=object)

In [0]:
vectores = []
for i in range(379):
    vec = ob[i] + fsu[i]
    vec = vec + ssu[i] 
    vectores.append(vec)

UFuncTypeError: ignored

In [0]:
KMO = MultiOutputClassifier(KNeighborsClassifier(n_neighbors = 11))
KMO.fit(XT, y)
preds = KMO.predict(test)

In [0]:
a0 = []
a1 = []
for pred in preds:
    a0.append(pred[0])
    a1.append(pred[1])

In [0]:
test['predicted_a0'] = a0
test['predicted_a1'] = a1
pred_df = test[['neg_entries', 'pos_entries','predicted_a0', 'predicted_a1']]
pred_df.to_csv('/drive/My Drive/Colab Notebooks/pred_DP.csv')